In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch_geometric

In [3]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


In [4]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


# Requirements

In [5]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

In [6]:
logger = logging.getLogger(__name__)

In [7]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
from src.optimization import MatrixFeatureBasedMDAClassifierTrainer, MatrixFeatureBasedMDAClassifierTester
from src.config import MatrixDecomposerConfig, SimpleClassifierConfig
from src.models import MDFeatureBasedMDAClassifier, MDFeatureBasedMDAClassifierFactory
from src.data import MicrobeDiseaseAssociationData, MicrobeDiseaseAssociationTrainTestSpliter
from src.features import get_associations, get_entities
from src.utils import train_test_sampler
from base import cross_validation, OptimizerConfig


2024-02-10 18:26:40,502 [INFO] NumExpr defaulting to 2 threads.


# Classification

## Data

In [9]:
associations = get_associations()

train_indices, test_indices = train_test_sampler(associations.shape[0], 0.7)

data = MicrobeDiseaseAssociationData(associations)

train_data = MicrobeDiseaseAssociationData(associations.iloc[train_indices])
test_data = MicrobeDiseaseAssociationData(associations.iloc[test_indices])

## Classifier

In [10]:
microbe_ids = get_entities().loc[get_entities()['type'] == 'Microbe']['id'].tolist()
disease_ids = get_entities().loc[get_entities()['type'] == 'Disease']['id'].tolist()

In [11]:
md_config = MatrixDecomposerConfig()
md_config.model_name = "PCA Decomposer for Feature Extraction"
md_config.microbe_ids = microbe_ids
md_config.disease_ids = disease_ids
md_config.n_components = 50
md_config.random_state = 1
md_config.decomposer = 'PCA'

In [12]:
simple_classifier_config = SimpleClassifierConfig()
simple_classifier_config.model_name = "simple classifier"
simple_classifier_config.input_dim = md_config.n_components * 2
simple_classifier_config.hidden_dim = 32
simple_classifier_config.output_dim = 1
simple_classifier_config.num_layers = 2
simple_classifier_config.dropout = 0.3

In [13]:
mda_classifier = MDFeatureBasedMDAClassifier(simple_classifier_config, md_config)

2024-02-10 18:26:41,723 [INFO] Initializing MDFeatureBasedMDAClassifier with model : simple classifier
2024-02-10 18:26:41,725 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-10 18:26:41,728 [INFO] Initial SimpleMLP with 100 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.3 dropout
2024-02-10 18:26:41,732 [INFO] Initializing MatrixFeatureExtractor
2024-02-10 18:26:41,735 [INFO] Initializing MFFeatureExtractor with model : None and decomposer : PCA


## Optimizer

In [14]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.optimizer = torch.optim.Adam
classifier_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
classifier_optimizer_config.lr = 0.01
classifier_optimizer_config.batch_size = 32
classifier_optimizer_config.n_epoch = 50
classifier_optimizer_config.exp_name = "adam optimizer"
classifier_optimizer_config.save = False
classifier_optimizer_config.save_path = None
classifier_optimizer_config.device = device
classifier_optimizer_config.report_size = 10  # batch to report ratio
classifier_optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [15]:
train_result = MatrixFeatureBasedMDAClassifierTrainer().train(model=mda_classifier,
                                                          data=train_data,
                                                          config=classifier_optimizer_config)

2024-02-10 18:26:41,756 [INFO] Call Training with adam optimizer
2024-02-10 18:26:41,766 [INFO] Calling build with associations :      disease  microbe  increased
481    44991    63336          0
632    66282    55403          0
561    13565    59219          0
764     1667    32606          0
591     7877    53656          0
..       ...      ...        ...
691    25026    54894          0
443    43621    32642          1
117    44112    49054          1
236    28016    35590          1
36     33293    58532          1

[628 rows x 3 columns]
2024-02-10 18:26:41,863 [INFO] interaction matrix with shape (5179, 5645) has built
2024-02-10 18:26:42,469 [INFO] mask matrix with shape (5179, 5645) has built. This matrix shows not non elements.
2024-02-10 18:26:45,382 [INFO] interaction has been imputed to delete nans
2024-02-10 18:26:53,498 [INFO] Initializing SimplePytorchData with X shape : torch.Size([628, 100]) and y shape : torch.Size([628, 1])
2024-02-10 18:26:53,501 [INFO] Running Sim

### Test

In [16]:
test_result = MatrixFeatureBasedMDAClassifierTester().test(model=mda_classifier,
                                                       data=test_data,
                                                       config=classifier_optimizer_config)

2024-02-10 18:26:57,314 [INFO] Call Testing with adam optimizer
2024-02-10 18:26:57,582 [INFO] Initializing SimplePytorchData with X shape : torch.Size([270, 100]) and y shape : torch.Size([270, 1])
2024-02-10 18:26:57,585 [INFO] Running Simple Tester with config : adam optimizer
2024-02-10 18:26:57,587 [INFO] moving data and model to cpu
2024-02-10 18:26:57,637 [INFO] Result on Test Data : {'AUC': 0.9764047410008779, 'ACC': 0.9259259259259259, 'F1 Score': 0.9258893280632411, 'AUPR': 0, 'Loss': 0.3327157033814324}


In [17]:
test_result.get_result()

{'AUC': 0.9764047410008779,
 'ACC': 0.9259259259259259,
 'F1 Score': 0.9258893280632411,
 'AUPR': 0,
 'Loss': 0.3327157033814324}

## Cross Validation

In [18]:
trainer = MatrixFeatureBasedMDAClassifierTrainer()
tester = MatrixFeatureBasedMDAClassifierTester()
factory = MDFeatureBasedMDAClassifierFactory(simple_classifier_config, md_config)
spliter = MicrobeDiseaseAssociationTrainTestSpliter(data.associations)
cross_validation(k=5, data_size=data.associations.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-10 18:26:57,673 [INFO] Initializing MDFeatureBasedMDAClassifierFactory
2024-02-10 18:26:57,678 [INFO] Initializing MicrobeDiseaseAssociationTrainTestSpliter
2024-02-10 18:26:57,680 [INFO] Start 5-fold Cross Validation with config : adam optimizer
2024-02-10 18:26:57,687 [INFO] ---- Fold 1 ----
2024-02-10 18:26:57,692 [INFO] Initializing MDFeatureBasedMDAClassifier with model : simple classifier
2024-02-10 18:26:57,696 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-10 18:26:57,699 [INFO] Initial SimpleMLP with 100 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.3 dropout
2024-02-10 18:26:57,702 [INFO] Initializing MatrixFeatureExtractor
2024-02-10 18:26:57,705 [INFO] Initializing MFFeatureExtractor with model : None and decomposer : PCA
2024-02-10 18:26:57,706 [INFO] Call Training with adam optimizer
2024-02-10 18:26:57,713 [INFO] Calling build with associations :      disease  microbe  increased
0      5